In [9]:
from langchain_groq import ChatGroq

api_key = "Your_API_KEY"

model = ChatGroq(model="qwen/qwen3-32b", groq_api_key=api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f026fec3310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f026fec28f0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_classic.schema import (
    AIMessage, 
    HumanMessage,
    SystemMessage
)

In [23]:
content = '''
    LangChain is an innovative framework that allows developers to build applications that can reason with language models. 
    By connecting LLMs with external tools, data sources, and custom logic, LangChain enables agents to perform complex tasks autonomously. 
    Whether you want to answer questions from documents, summarize text, or even automate workflows, LangChain provides a flexible interface to integrate natural language understanding with real-world actions. 
    Its modular design makes it easy to experiment with different models, prompts, and agents, allowing developers to focus on the unique behavior of their applications rather than low-level integrations.    
'''

content

'\n    LangChain is an innovative framework that allows developers to build applications that can reason with language models. \n    By connecting LLMs with external tools, data sources, and custom logic, LangChain enables agents to perform complex tasks autonomously. \n    Whether you want to answer questions from documents, summarize text, or even automate workflows, LangChain provides a flexible interface to integrate natural language understanding with real-world actions. \n    Its modular design makes it easy to experiment with different models, prompts, and agents, allowing developers to focus on the unique behavior of their applications rather than low-level integrations.    \n'

In [14]:
chat_message = [
    SystemMessage(content="You are an expert in summarizing the content"),
    HumanMessage(content=f"Please summarize the content and give a short and concise response:\nText: {speech}")
]

In [24]:
model.get_num_tokens(content)

140

In [ ]:
## Ways of summarizing the content
## Way 1
model.invoke(chat_message).content

'<think>\nOkay, let\'s see. The user wants a summary of the provided text about LangChain. First, I need to read through the text carefully to understand the main points.\n\nThe text starts by introducing LangChain as a framework for building apps that use language models. It mentions connecting LLMs with external tools, data sources, and custom logic. The purpose here is to enable agents to perform complex tasks autonomously. \n\nThen there are examples of tasks: answering questions from documents, summarizing text, automating workflows. The framework provides a flexible interface for integrating natural language understanding with real-world actions. \n\nThe last part talks about the modular design, which allows experimenting with different models, prompts, and agents. This modularity helps developers focus on their app\'s unique behavior without dealing with low-level integrations.\n\nNow, I need to condense this into a short, concise summary. The key elements are: LangChain is a fr

In [ ]:
## Way 2

from langchain_classic.schema.runnable import RunnableSequence, RunnableLambda
from langchain_classic import PromptTemplate

prompt = """
    Write a summary of the following content:
    content: {content}
    Translate the precise summary to {language}
    """

prompt_template = PromptTemplate(
    input_variables=['content', 'language'],
    template=prompt
)

prompt_template


PromptTemplate(input_variables=['content', 'language'], input_types={}, partial_variables={}, template='\n    Write a summary of the following content:\n    content: {content}\n    Translate the precise summary to {language}\n    ')

In [40]:
response = prompt.format(content=content, language="German")
print(response)

model.get_num_tokens(response)


    Write a summary of the following content:
    content: 
    LangChain is an innovative framework that allows developers to build applications that can reason with language models. 
    By connecting LLMs with external tools, data sources, and custom logic, LangChain enables agents to perform complex tasks autonomously. 
    Whether you want to answer questions from documents, summarize text, or even automate workflows, LangChain provides a flexible interface to integrate natural language understanding with real-world actions. 
    Its modular design makes it easy to experiment with different models, prompts, and agents, allowing developers to focus on the unique behavior of their applications rather than low-level integrations.    

    Translate the precise summary to German
    


174

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

# Step 1: Format the prompt using your inputs
prompt_runnable = RunnableLambda(
    lambda inputs: f"{inputs['content']}\nTranslate to {inputs['language']}"
)

# Step 2: Wrap ChatGroq call
llm_runnable = RunnableLambda(
    lambda prompt_text: model.invoke(prompt_text)  # just pass a string
)

# Step 3: Chain them
model_chain = RunnableSequence(prompt_runnable, llm_runnable)

# Step 4: Run the chain
inputs = {"content": content, "language": "Kannada"}
summary = model_chain.invoke(inputs)
print(summary)



content='<think>\nOkay, I need to translate the given English text about LangChain into Kannada. Let me start by reading through the original text to understand the key points. The main idea is that LangChain is a framework that allows developers to build applications using language models, connecting them with external tools, data sources, and custom logic. It enables agents to perform complex tasks autonomously, such as answering questions from documents, summarizing text, or automating workflows. The modular design allows for easy experimentation with different models, prompts, and agents.\n\nFirst, I\'ll break down each sentence and translate them individually, making sure to maintain the technical terms accurately. Words like "framework" can be translated as "ಪರಿಸರ" or "ಅನುವಾದ ಪರಿಸರ" depending on the context. "Innovative" might be "ಪ್ರತಿಭಟನಾತ್ಮಕ" or "ನವೀಕರಣಾತ್ಮಕ". "Natural language understanding" could be "ಸಹಜ ಭಾಷಾ ಅರ್ಥಮಟ್ಟ" or "ಸಹಜ ಭಾಷಾ ಅರ್ಥವನ್ನು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವುದು".\n\nI need to

In [63]:
## Stuff Documnetation chain

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load_and_split()

docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [65]:
template = '''
    Write a concise and short summary of the following speech,
    speech: {text}
'''

prompt = PromptTemplate(input_variables=['text'], template=template)
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    Write a concise and short summary of the following speech,\n    speech: {text}\n')

In [66]:
from langchain_classic.chains.summarize import load_summarize_chain

chain = load_summarize_chain(model, chain_type='stuff', prompt=prompt, verbose=True)
output_summary = chain.run(docs)
output_summary

/tmp/ipykernel_54720/3334489617.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  output_summary = chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Write a concise and short summary of the following speech,
    speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those livi ng abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
o

'<think>\nOkay, I need to summarize this speech by Dr. A.P.J. Abdul Kalam. Let me start by reading through it carefully. The speech is his farewell address after five years as the President of India. He mentions various key messages and experiences from his tenure.\n\nFirst, he talks about the aspirations of the youth, inspired by a question from a girl named Anukriti. Then there\'s a part about empowering villages and rural development through initiatives like PURA. He emphasizes agriculture\'s importance with the "Seed to Food" concept. \n\nHe shares stories of overcoming challenges, like the man with physical challenges who became a musician and the people in Jammu & Kashmir recovering from an earthquake. He also mentions the courage of the Car Nicobar islanders after the tsunami. \n\nTechnology and connectivity are highlighted, especially the Pan African e-Network project linking India and Africa. There\'s gratitude towards the defense forces and the youth movement for a developed 

In [67]:
## Map reduce technique to summarize the large documents

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
document_chunks = text_splitter.split_documents(docs)
document_chunks

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those livi ng abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealt

In [68]:
len(document_chunks)

13

In [70]:
prompts_for_chunks = '''
    Please summarize the below speech:
    Speech: '{text}'
    Summary:
'''

map_prompt_template = PromptTemplate(input_variables=['text'],
                                     template=prompts_for_chunks)
map_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template="\n    Please summarize the below speech:\n    Speech: '{text}'\n    Summary:\n")

In [71]:
final_prompt = '''
    Provide the final summary of the entire speech with these important points.
    Add a motivation title, start the precise summary with an introduction and provide the summary in number
    points for the sppech.
    Speech: {text}
'''

final_prompt_template = PromptTemplate(input_variables=['text'], template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\n    Provide the final summary of the entire speech with these important points.\n    Add a motivation title, start the precise summary with an introduction and provide the summary in number\n    points for the sppech.\n    Speech: {text}\n')

In [ ]:
summary_chain = load_summarize_chain(
    llm=model,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template
)

output = summary_chain.run(document_chunks)
output

In [ ]:
## Refine chain summarization

refined_summary_chain = load_summarize_chain(
    llm=model,
    chain_type='refine',
    verbose=True
)

output_summary = refined_summary_chain.run(document_chunks)
output_summary